Hopefully, This script will allow me to load up and fix up the moored data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
from scipy import io
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [2]:
saveloc='/ocean/kflanaga/MEOPAR/ORCAData'
Mooring='Hoodsport'

In [3]:
orca_dict=io.loadmat(f'{saveloc}/Hoodsport.mat')

Alright. I am going to say that Btemp=temperature, Bdepth=depth, Bfluor= chlorophyll fluorescence
Bsal=Salinity

In [4]:
newData=list(zip(orca_dict['Btime'][1],orca_dict['Bdepth'][1],orca_dict['Btemp'][1],
                 orca_dict['Bsal'][1],orca_dict['Bfluor'][1]))
columns=['Decimal_days','Z','Temperature','Salinity','Chlorophyll']
df=pd.DataFrame(newData,columns=columns)

In [5]:
# rounding down the decimal days so that all the data can be grouped together by day. 
YD_rounded=[]

for yd in df['Decimal_days']:
    if np.isnan(yd) == True:
        YD_rounded.append(float("NaN"))
    else:
        YD_rounded.append(math.floor(yd))
        
df['YD_rounded']=YD_rounded

In [6]:
#Averaging over daily values in order to better compare with the daily model resolution.
dfg=df.groupby(by='YD_rounded')
df_avg=dfg.mean()

In [14]:
Lon

-123.112583

In [7]:
#Creating a Column that contains datetimes. 
UTC=[]

for yd in df_avg['Decimal_days']:
    if np.isnan(yd) == True:
        UTC.append(float("NaN"))
    else:
        start = dt.datetime(1999,12,31)      
        delta = dt.timedelta(yd)     
        offset = start + delta
        time=offset.replace(microsecond=0)
        UTC.append(time)

In [8]:
#Adding new variables for model matching. 
df_avg['dtUTC']=UTC
Lat=47.421817
Lon=-123.112583
press=gsw.p_from_z(-1*df_avg['Z'],Lat)
df_avg['SA']=gsw.SA_from_SP(df_avg['Salinity'],press,
                       Lon,Lat)
df_avg['CT']=gsw.CT_from_t(df_avg['SA'],df_avg['Temperature'],press)

In [9]:
#Removing all variables that are outside of model date range. 
df_avg=df_avg[(df_avg.dtUTC >= dt.datetime(2007,1,1))&(df_avg.dtUTC < dt.datetime(2020,1,1))]
df_avg.reset_index(inplace=True)

In [12]:
##### Saving data as Pickle files to be used in the summary file
saveloc='/ocean/kflanaga/MEOPAR/ORCAData'
with open(os.path.join(saveloc,f'daily_data_{Mooring}.pkl'),'wb') as hh:
    pickle.dump(df_avg,hh)